In [1]:
import os
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"

In [3]:
filelist = os.listdir(path)
filelist.sort()
df = pd.DataFrame()
for file in tqdm(filelist[-300:]):
    df0 = pd.read_csv(os.path.join(path, file))
    df = pd.concat([df, df0], ignore_index=True)

100%|██████████| 300/300 [00:01<00:00, 172.93it/s]


In [30]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()

In [31]:
# 计算中轨线(MB)、上轨线(UB)和下轨线(LB)
def get_mb_ub_lb(df, length=20, K=2):
    df=df.copy()
    df['MB']=df['c'].rolling(length).mean() # 计算20日收盘价的简单移动平均线
    # 计算标准差
    df['seta']=np.sqrt((np.square(df['c']-df['MB'])).rolling(length).mean())
    df['UB']=df['MB']+K*df['seta']
    df['LB']=df['MB']-K*df['seta']
    return df['MB'],df['UB'],df['LB']

In [32]:
df1['MB'],df1['UB'],df1['LB'] = get_mb_ub_lb(df1)
df1=df1[39:]
df1

,t,o,h,l,c,v,MB,UB,LB
39,1671116400000,17448.0,17483.2,17355.0,17424.0,30874.038,17681.685,18008.779399,17354.590601
40,1671120000000,17424.0,17437.2,17330.0,17391.7,26907.949,17663.165,18007.761129,17318.568871
41,1671123600000,17391.7,17424.4,17367.2,17392.1,15494.334,17641.375,18002.603643,17280.146357
42,1671127200000,17392.1,17415.2,17310.0,17395.5,19325.547,17619.665,17993.629970,17245.700030
43,1671130800000,17395.5,17467.7,17383.0,17435.1,17252.568,17601.225,17980.578121,17221.871879
...,...,...,...,...,...,...,...,...,...
7195,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,27664.420,28088.952892,27239.887108
7196,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,27651.790,28077.471425,27226.108575
7197,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,27634.740,28059.336894,27210.143106
7198,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,27617.350,28041.848828,27192.851172


In [33]:
df1.to_csv('BBH.csv', index=False)